In [1]:
import numpy as np
import torch
from torch import Tensor
from gpytorch.kernels import ScaleKernel, RBFKernel
from linear_operator.operators import KroneckerProductLinearOperator

In [2]:
latent_dim = 2
index_dim = 1
n_C = 50
n_X = 20
noise_scale = 0.001

In [6]:
default_kernel_parameters = {'X_raw_outputscale': [0.0], 'X_raw_lengthscale': [[0.1 for _ in range(latent_dim)]],
                            'C_raw_outputscale': [0.9], 'C_raw_lengthscale': [[0.1 for _ in range(index_dim)]]}
C = Tensor(np.linspace(-10, 10, n_C))
X = Tensor(np.random.multivariate_normal([0 for _ in range(latent_dim)], np.eye(latent_dim), (n_X,)))

covar_module_X = ScaleKernel(RBFKernel(ard_num_dims=latent_dim))
covar_module_C = ScaleKernel(RBFKernel(ard_num_dims=index_dim))

covar_module_X.raw_outputscale.data = Tensor(default_kernel_parameters['X_raw_outputscale'])
covar_module_X.base_kernel.raw_lengthscale.data = Tensor(default_kernel_parameters['X_raw_lengthscale'])
covar_module_C.raw_outputscale.data = Tensor(default_kernel_parameters['C_raw_outputscale'])
covar_module_C.base_kernel.raw_lengthscale.data = Tensor(default_kernel_parameters['C_raw_lengthscale'])

covar_X = covar_module_X(X)
print(covar_X.shape)
covar_C = covar_module_C(C)
print(covar_C.shape)
# K + sigma^2 * I 

torch.Size([20, 20])
torch.Size([50, 50])


In [9]:
covar_final = KroneckerProductLinearOperator(covar_X, covar_C)

In [10]:
covar_final.to_dense()

RuntimeError: The expected shape of the kernel was torch.Size([20, 20]), but got torch.Size([1, 20, 20]). This is likely a bug in GPyTorch.